In [1]:

# Import necesary libraries
import yfinance as yf
import datetime as dt
from stocktrends import Renko

################################PLEASE READ ME####################################
#Stocktrends' author has renamed get_bricks() function to get_ohlc_data()
#therefore you may get error when trying to run line 43 below
#if that is the case please comment out line 43 and remove # sign from line 44 and rerun
##################################################################################

# Download historical data for required stocks
ticker = "AAPL"
ohlcv = yf.download(ticker,dt.date.today()-dt.timedelta(1825),dt.datetime.today())

def ATR(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2

def renko_DF(DF):
    "function to convert ohlc data into renko bricks"
    df = DF.copy()
    df.reset_index(inplace=True)
    df = df.iloc[:,[0,1,2,3,5,6]]
    df.rename(columns = {"Date" : "date", "High" : "high","Low" : "low", "Open" : "open","Adj Close" : "close", "Volume" : "volume"}, inplace = True)
    df2 = Renko(df)
    df2.brick_size = round(ATR(DF,120)["ATR"][-1],0)
    renko_df = df2.get_ohlc_data() #if using older version of the library please use get_bricks() instead
    return renko_df


renko_data = renko_DF(ohlcv)

[*********************100%***********************]  1 of 1 completed


/opt/anaconda3/lib/python3.8/site-packages/stocktrends/indicators.py:58: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  self.cdf.loc[0, 1:] = [close - brick_size, close, close - brick_size, close]


In [2]:
renko_data

,date,open,high,low,close,uptrend
0,2016-06-01,18.0,21.0,18.0,21.0,True
1,2016-07-27,21.0,24.0,21.0,24.0,True
2,2016-09-15,24.0,27.0,24.0,27.0,True
3,2017-02-01,27.0,30.0,27.0,30.0,True
4,2017-03-01,30.0,33.0,30.0,33.0,True
...,...,...,...,...,...,...
97,2021-04-07,123.0,126.0,123.0,126.0,True
98,2021-04-08,126.0,129.0,126.0,129.0,True
99,2021-04-09,129.0,132.0,129.0,132.0,True
100,2021-05-11,129.0,129.0,126.0,126.0,False
